# Script to Find Out Which Sessions Enable The DotnetRunTime Provider 

In [ ]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.Text.RegularExpressions;

In [ ]:
public sealed class EventTraceProvider
{
    public string Name { get; }
    public string Guid { get; }
    public string Type { get; }
    public string Status { get; }

    public EventTraceProvider(string name, string guid, string type, string status)
    {
        Name = name;
        Guid = guid;
        Type = type;
        Status = status;
    }
}

In [ ]:
public static void Run()
{
    string logmanOutput = RunLogmanCommand("query", "-ets");

    // Parse and display the data
    List<EventTraceProvider> providers = ParseLogmanOutput(logmanOutput);

    foreach (EventTraceProvider provider in providers)
    {
        //if (provider.Name.StartsWith("PerfView") || provider.Name.StartsWith("AppModel"))
        {

            Console.WriteLine($"Provider Name: {provider.Name}");

            // Retrieve and parse details for the provider
            string providerDetailsOutput = RunLogmanCommand("query", "-ets -n " + provider.Name);
            //Dictionary<string, string> providerDetails = ParseProviderDetails(providerDetailsOutput, args[0]);
            string providerDetails = ParseProviderDetails(providerDetailsOutput);
            if (providerDetails != null) 
            {
                Console.WriteLine(providerDetails);
            }

            Console.WriteLine();
        }
    }

}

public static string RunLogmanCommand(string action, string arguments)
{
    try
    {
        // Create process start info
        ProcessStartInfo psi = new ProcessStartInfo();
        psi.FileName = "logman";
        psi.Arguments = $"{action} {arguments}";
        psi.RedirectStandardOutput = true;
        psi.UseShellExecute = false;

        // Start the process
        Process process = Process.Start(psi);

        // Read the output
        string output = process.StandardOutput.ReadToEnd();

        // Close the process
        process.WaitForExit();
        process.Close();

        return output;
    }
    catch (Exception ex)
    {
        Console.WriteLine("Error: " + ex.Message);
        return null;
    }
}


public static List<EventTraceProvider> ParseLogmanOutput(string logmanOutput)
{
    List<EventTraceProvider> providers = new List<EventTraceProvider>();

    string[] lines = logmanOutput.Split(new[] { Environment.NewLine }, StringSplitOptions.None);

    foreach (string line in lines)
    {
        int providerEnd = line.IndexOf("Trace                         Running");
        if (providerEnd != -1)
        {
            string providerName = line.Substring(0, providerEnd).TrimEnd();
            //Console.WriteLine("---{0}------", providerName);
            providers.Add(new EventTraceProvider(providerName, null, null, null));
        }
    }

    return providers;
}

public static string ParseProviderDetails(string providerDetailsOutput)
{
    Dictionary<string, string> details = new Dictionary<string, string>();

    string[] lines = providerDetailsOutput.Split(new[] { Environment.NewLine }, StringSplitOptions.None);

    foreach (string line in lines)
    {
        if (line.StartsWith("Name:                 Microsoft-Windows-DotNETRuntime"))
        {
            return providerDetailsOutput;
        }
    }

    return null;
}

In [ ]:
Run()

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id
#!about